# Introduction

In this notebook, I will implement the K-Nearest Neighbor algorithm from scratch and test my implementation on scikit-learn's breast cancer dataset.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Model Implementation

Now let's build our KNN classifier. Essentially, the model will look at the k nearest samples and see which class they belong to and assign the label of current sample to the most frequently appeared class. 

To accomplish this, I will predict an input in two steps:

1. compute the distances of the input to all other samples with known labels
2. assign a class label to the input based on the most frequent class label of the k cloest samples

In [2]:
eps = 1e-8

**Predict**

In [3]:
def predict(X_test, k):
    distances = compute_distance(X_test)
    return predict_labels(distances, k)

**Compute the distance of input to all other samples**

In [4]:
def compute_distance(X_test):
    X_test_squared = np.sum(X_test ** 2, axis=1, keepdims=True)
    X_train_squared = np.sum(X_train ** 2, axis=1, keepdims=True)
    two_X_test_X_train = np.dot(X_test, X_train.T)

    return np.sqrt(eps + X_test_squared - 2 * two_X_test_X_train + X_train_squared.T)

**Assign a class label based on the k nearest samples**

In [5]:
def predict_labels(distances, k):
    num_test = distances.shape[0]
    y_pred = np.zeros(num_test)
    
    for i in range(num_test):
        y_indices = np.argsort(distances[i, :])
        k_closest_classes = y_train[y_indices[: k]].astype(int)
        y_pred[i] = np.argmax(np.bincount(k_closest_classes))

    return y_pred

# Test

Now we have implementated the algorithm, let's test our model based on scikit-learn's breast cancer dataset. First I will apply some preprocessing and split the train and test sets.

In [6]:
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X = load_breast_cancer()['data']
y = load_breast_cancer()['target']
scaler = MinMaxScaler(feature_range=(-1, 1))
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=42)

**Final the best K**

In [7]:
for k in range(2, 11):
    y_pred = predict(X_train, k)
    print(accuracy_score(y_train, y_pred))

0.9812206572769953
0.9859154929577465
0.9788732394366197
0.9812206572769953
0.9765258215962441
0.9788732394366197
0.9765258215962441
0.9812206572769953
0.9788732394366197


By using different k to predict the training labels, we can see that with k = 3 the model has best performance. Now let's see its accuracy on the test set:

In [8]:
final_y_pred = predict(X_test, 3)
print(accuracy_score(y_test, final_y_pred))

0.972027972027972


To better understand this accuracy, I also used scikit-learn's built in K-Nearest Neighbor Classifier with k = 3 to predict the test set. The output is as follows:

In [9]:
from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, y_train)
y_pred = neigh.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.972027972027972


The accuracy of scikit-learn's model is identical to our model, which thus shows our implementation is successful.